In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

# 已定义的模块
from model.CNN_feature_extractor import CNNFeatureExtractor
from model.model import TemporalLSTM
from dataset import ImageSequenceDataset 


In [4]:
# 图像预处理
import os
from pathlib import Path
from model.model import OzonePredictor


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 超参数
# 提供根目录
notebook_path = Path().resolve().parent.parent
data_dir = notebook_path / "data"
json_path = data_dir / "dataset.json"
json_path = os.path.normpath(json_path)

batch_size = 4
num_epochs = 20
lr = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 数据加载
dataset = ImageSequenceDataset(json_path)
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=0,shuffle=True)

# 模型构建
model = OzonePredictor(cnn_out_dim=512, lstm_hidden_dim=512, output_dim=849)
model.to(device)

# 优化器与损失函数
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 训练循环
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        imgs = batch['images'].to(device)     # (B, T, 4, C, H, W)
        # print(imgs.shape, flush=True)       # torch.Size([4, 3, 4, 3, 224, 224])
        npy = batch['npy'].to(device)    # (B, T) or (B, output_dim)
        # print(npy.shape, flush=True)       # torch.Size([4, 3, 4, 3, 224, 224])

        optimizer.zero_grad()
        outputs = model(imgs)               # (B, output_dim)
        print("output shape:",outputs.shape,flush=True)
        print("npy shape:",npy.shape,flush=True)
        loss = criterion(outputs, npy)
        print("loss:",loss,flush=True)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

output shape: torch.Size([4, 3, 849])
npy shape: torch.Size([4, 3, 849])
loss: tensor(546.3868, device='cuda:0', grad_fn=<MseLossBackward0>)
output shape: torch.Size([4, 3, 849])
npy shape: torch.Size([4, 3, 849])
loss: tensor(2844.3350, device='cuda:0', grad_fn=<MseLossBackward0>)
output shape: torch.Size([4, 3, 849])
npy shape: torch.Size([4, 3, 849])
loss: tensor(802.9528, device='cuda:0', grad_fn=<MseLossBackward0>)
output shape: torch.Size([4, 3, 849])
npy shape: torch.Size([4, 3, 849])
loss: tensor(861.6654, device='cuda:0', grad_fn=<MseLossBackward0>)
output shape: torch.Size([4, 3, 849])
npy shape: torch.Size([4, 3, 849])
loss: tensor(1229.5194, device='cuda:0', grad_fn=<MseLossBackward0>)
output shape: torch.Size([4, 3, 849])
npy shape: torch.Size([4, 3, 849])
loss: tensor(800.1630, device='cuda:0', grad_fn=<MseLossBackward0>)
output shape: torch.Size([4, 3, 849])
npy shape: torch.Size([4, 3, 849])
loss: tensor(992.4319, device='cuda:0', grad_fn=<MseLossBackward0>)
output shap

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>